# Brain Image Library

Name: Natalie Pham

Andrew ID: thanhngp

* You can work in groups, it is highly encouraged. Basic divide-and-conquer.
* But each of you will have to submit your notebook and data on Canvas.

You have been given access to a database where you can pull information from one table. The purpose of this exercise is to

* clean the local copy of the table
* compute some basic information about the files in the table
* add these features to the table
* use the original raw data to create a second table
* use these 2 tables to make some pretty plots

In theory, the file and dataset level features you will be computing in this exercise will be stored in a database and used for a dashboard.

## Exercise 1
a. The variable `file` belows points to a CSV file with the file informations. Load the table into the workspace using Pandas or Dask.

**Hint**
If you use Pandas, set `low_memory` to `False`. For more info click [here](https://stackoverflow.com/questions/58551446/how-to-set-low-memory-to-false).

In [ ]:
!pip install tqdm
!pip install pathlib
!pip install warnings

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import time
from pathlib import Path
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# INSERT CODE HERE
file = '/bil/workshops/2022/data-science/final_project_dataset.csv'
def load_data( file ):
    t1 = time.time()
    df = pd.read_csv(file, low_memory=False, sep=',' )
    t2 = time.time()
    total = t2-t1
    print(total)
    return df

df = load_data(file)

20.26436424255371


In [ ]:
print(df.shape)
df.head()

(2825872, 13)


,dataset_uuid,collection_id,dataset_id,sample_id,directory,filepath,filename,file_extension,file_size,file_creation_date,sha256,md5,xxh128
0,NaN,009c1e6fcc03ebac,UNKNOWN,190895,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190895-metainfo.txt,.txt,693.0,2019-12-0503:26:25,2f83f9da49a4fc59b80b9185207055832653ddf51117e3...,aae6752c964e56146934fef4e2e21491,None
1,NaN,009c1e6fcc03ebac,UNKNOWN,190895,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190895.json,.json,534.0,2019-12-0503:27:38,55b2e61954ae32118f7d923a710489b0355123142579b9...,2704360b304105e180fb08ef06d1a52b,None
2,NaN,009c1e6fcc03ebac,UNKNOWN,190895,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,xxhash_dat_mouseID_19022522-190895,NaN,1071128.0,2019-12-0408:54:50,906545fa0dead8e6a5496ba803e9acd2a2d439b66db691...,fa2aaf565457ed5d5701d22d59f54506,None
3,NaN,009c1e6fcc03ebac,UNKNOWN,191178,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032...,xxhash_dat_mouseID_19032517-191178,NaN,1096722.0,2020-06-2907:43:20,c4e07196782566e7773f69b1eba310492c2bd5355a273f...,18758836aa7a3697dfcbc10ffc76dba2,None
4,NaN,009c1e6fcc03ebac,UNKNOWN,190896,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190896.json,.json,534.0,2020-06-2204:21:19,cbca27551e57bba24c13aff4f8344614b5ff95fb2ac7c0...,dd486c5107c9ee689df7588eedf399f6,None


b. Print the column names.

In [ ]:
# INSERT CODE HERE
df.keys()

Index(['dataset_uuid', 'collection_id', 'dataset_id', 'sample_id', 'directory',
       'filepath', 'filename', 'file_extension', 'file_size',
       'file_creation_date', 'sha256', 'md5', 'xxh128'],
      dtype='object')

c. This dataframe has one line per file in the file system. The data wrangler already included some file level statistics in the table. However before we can proceed adding more columns, we need to fix some things.

For example, if you take a look at the file creation dates, these look like

```
df['file_creation_date'][0]
'2019-12-0503:26:25'
```

when in fact these should look like 

```
'2019-12-05 03:26:25'
```

First change the values in the column `file_creation_date` so the dates are correct.

Second, change the data type of this series to be a `datetime` object.

**Hint**
* Add a whitespace.

In [ ]:
# INSERT CODE HERE
def time_formatting(datetime):
    # datetime(str): 2019-12-0503:26:25
    
    # Output:
    # new_datetime: fixed format for date/time with proper spacing (2019-12-05 03:26:25)
    
    datetime = str(datetime)
    new_datetime = datetime[0:10] + ' ' + datetime[10:]
    return new_datetime

start = time.time()

print('Computing file_creation_date...')

temp_file = 'manifest.tsv'

if Path(temp_file).exists()==True:
    df = pd.read_csv( temp_file, sep='\t', low_memory=False )
else:
    df['file_creation_date'] = df.apply(lambda d: time_formatting(d.file_creation_date), axis=1)
    df['file_creation_date'] = pd.to_datetime(df['file_creation_date'],errors='coerce')

end = time.time()
print('Time taken: ', end-start)

Computing file_creation_date...
Time taken:  20.99143695831299


In [ ]:
print(df['file_creation_date'].dtypes)

datetime64[ns]


d. The column `file_size` has the file size in bytes. Add a column named `human_readable_file_size` that is dtype string. This string representation of the file size should be an approximation to the nearest unit with one decimal point. For example, `15M`, `1.7G` and `5.6T`.

In [ ]:
# INSERT CODE HERE
# add a new column 'human_readable_file_size'
df['human_readable_file_size'] = ''

def file_size_conversion(size):
    # Input: 
    # size (int): file size in bytes
    
    # Output:
    # r_size (str): approximate human readable file size (KB, MB, GB, TB)
    
    if np.isnan(size) == True or size < 0:
        r_size = ''
    elif size < 10**2: # 0 - 99 bytes
        r_size = str(round(size,1)) + 'B'
    elif size >= 10**2 and size < 10**5: # 100 - 99,999 bytes
        r_size = str(round(size/(10**3), 1)) + 'K'
    elif size >= 10**5 and size < 10**8: # 100,000 - 99,999,999 bytes
        r_size = str(round(size/(10**6), 1)) + 'M'
    elif size >= 10**8 and size < 10**11: # 100,000,000 - 99,999,999,999 bytes
        r_size = str(round(size/(10**9), 1)) + 'G'
    else: # above 100,000,000,000 bytes
        r_size = str(round(size/(10**12), 1)) + 'T'
    
    return(r_size)

start = time.time()

print('Computing human_readable_file_size...')

temp_file = 'manifest.tsv'

if Path(temp_file).exists()==True:
    df = pd.read_csv( temp_file, sep='\t', low_memory=False )
else:
    df['human_readable_file_size'] = df.apply(lambda s: file_size_conversion(s.file_size), axis=1)

end = time.time()
print('Time taken: ', end-start)

Computing human_readable_file_size...
Time taken:  21.508914470672607


In [ ]:
df.head(20)

,dataset_uuid,collection_id,dataset_id,sample_id,directory,filepath,filename,file_extension,file_size,file_creation_date,sha256,md5,xxh128,human_readable_file_size,exists
0,NaN,009c1e6fcc03ebac,UNKNOWN,190895,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190895-metainfo.txt,.txt,693.0,2019-12-05 03:26:25,2f83f9da49a4fc59b80b9185207055832653ddf51117e3...,aae6752c964e56146934fef4e2e21491,None,0.7K,True
1,NaN,009c1e6fcc03ebac,UNKNOWN,190895,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190895.json,.json,534.0,2019-12-05 03:27:38,55b2e61954ae32118f7d923a710489b0355123142579b9...,2704360b304105e180fb08ef06d1a52b,None,0.5K,True
2,NaN,009c1e6fcc03ebac,UNKNOWN,190895,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,xxhash_dat_mouseID_19022522-190895,NaN,1071128.0,2019-12-04 08:54:50,906545fa0dead8e6a5496ba803e9acd2a2d439b66db691...,fa2aaf565457ed5d5701d22d59f54506,None,1.1M,True
3,NaN,009c1e6fcc03ebac,UNKNOWN,191178,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032...,xxhash_dat_mouseID_19032517-191178,NaN,1096722.0,2020-06-29 07:43:20,c4e07196782566e7773f69b1eba310492c2bd5355a273f...,18758836aa7a3697dfcbc10ffc76dba2,None,1.1M,True
4,NaN,009c1e6fcc03ebac,UNKNOWN,190896,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190896.json,.json,534.0,2020-06-22 04:21:19,cbca27551e57bba24c13aff4f8344614b5ff95fb2ac7c0...,dd486c5107c9ee689df7588eedf399f6,None,0.5K,True
5,NaN,009c1e6fcc03ebac,UNKNOWN,190896,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190896-metainfo.txt,.txt,693.0,2020-06-22 04:20:03,10d223a849f1c248c7c69a581bad3dc060c842bb40612c...,7c6f929ca4f45fc98d4689dcb625f75c,None,0.7K,True
6,NaN,009c1e6fcc03ebac,UNKNOWN,190896,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,xxhash_dat_mouseID_19022521-190896,NaN,992409.0,2020-06-22 04:30:48,0b4a31eadf12417698c9581d44849588b28b231986c3c8...,bc7badfacf028e7c898c04ddb0d19493,None,1.0M,True
7,NaN,009c1e6fcc03ebac,UNKNOWN,190896,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,readme.txt,.txt,51.0,2020-06-22 04:12:15,2d74dc07c14c812e0f67039fbe1d2a0e4891aaab687e37...,c5d0dfcd8110324b88357c1862504b8d,None,51.0B,True
8,NaN,009c1e6fcc03ebac,UNKNOWN,191180,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032...,xxhash_dat_mouseID_19032516-191180,NaN,1107522.0,2020-06-29 07:53:40,388f1184d4f622e848d1c9409f7c269728b13a393a97cd...,8b5bd3e47273082d596de03ffbb1513b,None,1.1M,True
9,NaN,009c1e6fcc03ebac,UNKNOWN,190897,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19022...,190897.json,.json,534.0,2020-06-22 05:07:25,30b735a014794e5fa018a07a39e3abb13c3e7537cc0e6f...,f8970d1ecfc5b6b793579e80ba08acd7,None,0.5K,True


e. The column `xxh128` is empty. To compute this hash and populate this column we will use the function below. However, due to time constraints, we will only compute these hash for files whose extensions are not `.tif`/`.tiff`.

For each file in the table, whose extension is not `.tif`/`.tiff`, compute the hash and store it in the column `xxh128`. Files without this hash, should remain as `None`.

In [ ]:
#INSERT CODE HERE
import subprocess

def compute_xxh128sum( filename ):
    if Path(str(filename)).is_file():
        # print(filename)
        results = subprocess.check_output('/bil/packages/xxhash/0.8.0/xxh128sum ' + str(filename) + ' | cut -d" " -f1 | xargs', shell=True,stderr=subprocess.DEVNULL)
        return results.decode("utf-8").strip

temp_file = 'manifest.tsv'

if Path(temp_file).exists()==True:
    df = pd.read_csv( temp_file, sep='\t', low_memory=False )
    
start = time.time()

print('Computing xxh128...')

df1 = df.loc[(df['file_extension'].str.contains('.tif')!=True)]
xxh128 = df1.loc[(df['xxh128'] == 'None')]
print(xxh128.shape)
# Originally the column 'xxh128' has 1109984 values are None and their file extension are not '.tif'
# I have able to compute ~ 9000 new values for 'xxh128' 

counter = 0
if df2.shape[0] > 0:
    for i in tqdm(xxh128.index): 
        # compute hash and store to 'xxh128' if file extension is not '.tif' or '.tiff'
        df['xxh128'][i] = compute_xxh128sum(df['filepath'][i])

        # checkpoint to save df to a temporary tsv file
        if counter % 1000 == 0 or i == len(xxh128.index)-1:
            #print('checkpoint at ', i)
            df['xxh128'].to_csv('temp1e.tsv', sep="\t",index=False)
            counter = counter + 1

end = time.time()
print('Time taken: ', end-start)

Computing xxh128...


100%|██████████| 2825872/2825872 [2:24:06<00:00, 326.84it/s]  

Time taken:  8646.156700611115


In [ ]:
print(df['xxh128'].value_counts())

f. Add column `exists`. Use the value in `filepath` to find if the file exists on disk. If the file exists, then populate the column as `True`. `False` otherwise.

**Hint** 
* Use `Path` from `pathlib`.
* All files should exist, if some of these don't, then report it. Not your fault.

In [ ]:
#INSERT CODE HERE
from pathlib import Path

def check_filepath(filepath):
    try:
        exist = Path(str(filepath)).exists()
        return(exist)
    except PermissionError:
        exist = False
        return(exist)

start = time.time()

temp_file = 'manifest.tsv'

print('Computing file_exists...')

if Path(temp_file).exists():
    df = pd.read_csv( temp_file, sep='\t', low_memory=False )
else:
    # add column 'exist'
    df['exists'] = None

for i in tqdm(df.index):
    # if np.isnan(df['exists'][i]) == True:
    if df['exists'][i] == None:
        df['exists'][i] = check_filepath(df['filepath'][i])
    
    # checkpoint to save df to a temporary tsv file
    if i % 5000 == 0 or i == len(df.index)-1:
        # print('checkpoint at ', i)
        df.to_csv('temp1f.tsv', sep="\t",index=False)

end = time.time()
print('Time taken: ', end-start)

Computing file_exists...


100%|██████████| 2825872/2825872 [2:02:17<00:00, 385.14it/s]  

Time taken:  7358.21319270134


In [ ]:
# output the number of files exist on disk vs. non-exist
df['exists'].value_counts()

True     2824340
False       1532
Name: exists, dtype: int64

g. Add columns `download_link`, `download_ready`, `response_code`, `download_timestamp`. Add these four columns and set the default values to `None`.

In [ ]:
#INSERT CODE HERE
df['download_link'] = None
df['download_ready'] = None
df['response_code'] = None
df['download_timestamp'] = None

h. We can populate the columns above at the same time to minimize the number of requests. In this case, each value in `filepath` can be turned into an download link. For example, the file `/bil/data/00/9c/009c1e6fcc03ebac/mouseID_19032506-191184/readme.txt` can be turned into a URL by replacing the prefix `/bil/data` with `https://download.brainimagelibrary.org`, leading to `https://download.brainimagelibrary.org/00/9c/009c1e6fcc03ebac/mouseID_19032506-191184/readme.txt`.

Download the response header (not the file) to determine if the file is reachable. Add `True` to `download_ready` if the file is reachable. `False` otherwise.

Save the response code from the request to column `response_code` as an integer, e.g. 202, 404, etc.

Record the timestamp when you made this request to column `download_timestamp`. This column should be of dtype `datetime`.

**Hints**
* Write a single method that can perform these tasks in a single call.
* You could also write a method that takes the full dataframe and returns and updated dataframe.
* You should be saving checkpoints to avoid recomputation.
* All links should be reachable. If some of these aren't that's a problem, but not your problem.



In [ ]:
#INSERT CODE HERE
import requests
import datetime

def check_download(filepath):
    path = str(filepath)[9:]
    # print(path)
    url = 'https://download.brainimagelibrary.org' + path
    response = requests.head(url)
    request_time = str(datetime.datetime.now())
    code = response.status_code
    if code == 200 or code == 308:
        download_ready = True
    else:
        download_ready = False
    
    return(url, download_ready, code, request_time)

start = time.time()

temp_file = 'manifest.tsv'

print('Computing download_ready...')
if Path(temp_file).exists():
    df = pd.read_csv( temp_file, sep='\t', low_memory=False )

for i in tqdm(df.index):    
    # get download link and check if link is ready for download
    if str(df['response_code'][i]) == 'nan':
        url, download_ready, code, request_time = check_download(df['filepath'][i])
        df['download_link'][i] = url
        df['download_ready'][i] = download_ready
        df['response_code'][i] = code
        df['download_timestamp'][i] = request_time
    
    # checkpoint to save df to a temporary tsv file
    if i % 10000 == 0 or i == len(df.index)-1:
        # print('checkpoint at ', i)
        df[['download_link','download_ready','response_code','download_timestamp']].to_csv('temp1h.tsv', sep="\t",index=False)

df['download_timestamp'] = pd.to_datetime(df['download_timestamp'],errors='coerce')
end = time.time()
print('Time taken: ', end-start)

Computing download_ready...


 76%|███████▋  | 2156373/2825872 [10:31:48<29:54:06,  6.22it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 77%|███████▋  | 2167493/2825872 [11:03:03<40:09:13,  4.55it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 77%|███████▋  | 2179564/2825872 [11:31:18<24:14:35,  7.41it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set t

In [ ]:
df['response_code'].value_counts()

i. Populate the column `dataset_uuid`. This one is tricky. This dataframe has one row per file. These files are grouped together in datasets. However the column `dataset_uuid` is empty. And now we need to populate it.

A dataset can be identified as a combination of `collection_id`, `sample_id` and `directory`. Generate a unique [UUID](https://www.educba.com/python-uuid/) for each unique combination of `collection_id`, `sample_id` and `directory`. Use the UUID to populate the column `dataset_uuid`. Keep in mind there will be multiple rows in this table that will share the combination of `collection_id`, `sample_id` and `directory`, hence all of these rows should also have the `dataset_uuid` as it means these are all part of the same dataset.

In [ ]:
import uuid

print('Compute uuid...')

temp_file = 'manifest.tsv'

if Path(temp_file).exists():
    df = pd.read_csv( temp_file, sep='\t', low_memory=False )
else:
    df['dataset_uuid'] = df.groupby(['collection_id', 'sample_id', 'directory']).dataset_uuid.transform(lambda f: uuid.uuid4())

k. If you haven't done it (and you should have saved some checkpoints), then save the df to disk as a pickle file and as a tsv file. Save it to `manifest.tsv` and `manifest.pkl`.

In [ ]:
print(df.shape)
df.head()

In [ ]:
#INSERT CODE HERE
# save to .tsv file
df.to_csv('manifest.tsv', sep="\t",index=False)

# save to .pkl file
import pickle
with open('manifest.pkl', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Exercise 2
a. Create another dataframe with columns `['dataset_uuid', 'collection_id', 'dataset_id', 'sample_id', 'directory', 'file_extensions','timestamp']`.

i. Set `file_extensions` to `None`.

ii. Use the values in the first dataframe to populate the other columns.

iii. Record the timestamp when you made this request to column `timestamp`. This column should be of dtype `datetime`.

**Hints**
* The first dataframe had a row per each file in the dataset. Whereas the second dataframe should have one row per dataset.

In [ ]:
#INSERT CODE HERE
import datetime

df2 = df[['dataset_uuid', 'collection_id', 'dataset_id', 'sample_id', 'directory']]
df2['file_extensions'] = None
df2['timestamp'] = datetime.datetime.now()

df2.drop_duplicates(subset=['dataset_uuid'], inplace=True)
print(df2.shape)

df2.head()

b. For each dataset in the first dataframe, count the number of file extensions.

i. Store these values as JSON in the column `file_extensions`. For example, if a dataset has 100 `.tif`s and 1000 `.jpeg`s, then you should store the string `{'tif':100,'jpeg':1000 }`.

ii. If a dataset has a file extension that is `None` or `Nan` store it as `other`. For example, `{'tif':100,'jpeg':1000, 'other':4 }`.

In [ ]:
#INSERT CODE HERE

def check_null(s):
    if str(s) == 'nan' or s==None:
        return True
    else:
        return False

temp_file = 'datasets.tsv'

if Path(temp_file).exists():                         
    df2 = pd.read_csv(temp_file, sep='\t', low_memory=False)
else:
    # if temp_file does not exist, populate df2['file_extensions']
    exts = []
    for i in tqdm(df['dataset_uuid'].unique()):
        #print(i)
        df_temp = df[df['dataset_uuid']==i]
        df_temp['file_extension'] = df_temp['file_extension'].apply(lambda f: 'other' if check_null(f) else f)
        extensions = str(df_temp['file_extension'].value_counts().to_json())
        #print(extensions)
        exts.append(extensions)
    df2['file_extensions'] = exts

In [ ]:
print(df2.shape)
df2.head()

c. Save the dataframe to disk as a pickle file and as a tsv file. Save it to `datasets.tsv` and `datasets.pkl`.

In [ ]:
#INSERT CODE HERE
# save to .tsv file
df2.to_csv('datasets2.tsv', sep="\t",index=False)

# save to .pkl file
import pickle
with open('datasets2.pkl', 'wb') as handle:
    pickle.dump(df2, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Exercise 3
a. Create another dataframe with columns `['timestamp', 'number_of_datasets', 'number_of_collections', 'number_of_files', 'file_extensions']`.

i. Timestamp records the current date, not the time, e.g. 2022-01-23.

ii. The number of datasets in the `datasets` dataframe goes in `number_of_datasets`.

iii. The number of unique collections in the `datasets` dataframe goes in `number_of_collections`.

iv. The total number of files in the `manifest` dataframe goes in `number_of_files`.

v. Another tricky one. Aggregate all the file extensions in column `file_extensions` in dataframe `datasets`, collect these as a single JSON block and it to `file_extensions` in this dataframe.

In [ ]:
#INSERT CODE HERE
temp_file = 'info.tsv'

if Path(temp_file).exists():                         
  df3 = pd.read_csv(temp_file, sep='\t', low_memory=False)
else:
  # Get timestamp
  tsp = str(datetime.datetime.now())[0:10]
  print(tsp)

  # Get number of datasets
  nDatasets = df2['dataset_uuid'].nunique()
  print(nDatasets)

  # Get number of collections
  nCollections = df2['file_extensions'].nunique()
  print(nCollections)

  # Get number of files
  nFiles = df['filepath'].nunique()
  print(nFiles)

  # Get all file_extensions
  df['file_extension'] = df['file_extension'].apply(lambda f: 'other' if check_null(f) else f)
  Exts = str(df['file_extension'].value_counts().to_json())
  print(Exts)

  # assign values to lists.  
  data = [{'timestamp':tsp, 'number_of_datasets':nDatasets, 'number_of_collections':nCollections, 'number_of_files':nFiles, 'file_extensions':Exts}]  
  
  # create DataFrame.  
  df3 = pd.DataFrame(data)  

  # convert type of 'timestamp' to type datetime
  df3['timestamp'] = pd.to_datetime(df3['timestamp'],errors='coerce')

In [ ]:
print(df3.shape)
df3.head()

b. Save the dataframe to disk as a pickle file and as a tsv file. Save it to `info.tsv` and `info.pkl`.

In [ ]:
#INSERT CODE HERE
# save to .tsv file
df3.to_csv('info.tsv', sep="\t",index=False)

# save to .pkl file
import pickle
with open('info.pkl', 'wb') as handle:
    pickle.dump(df3, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Exercise 4
Create some pretty plots and embed them in this notebook. Feel free to use any plotting library in Python. Make them pretty though. 

a. Waffleplot. Create a waffleplot from the column `file_extensions` in the dataframe `info`. 

* Title should be the date in `timestamp`.
* No axis labels.
* Add legend.

**Hints***
* Use `pywaffle`.

In [ ]:
#INSERT CODE HERE
import matplotlib.pyplot as plt
from pywaffle import Waffle
import json

temp_file = 'info.tsv'

if Path(temp_file).exists():                         
    df3 = pd.read_csv(temp_file, sep='\t', low_memory=False)

# json need to convert to dataframe for plot     
string = df3['file_extensions'][0]

# convert string to  object
d = json.loads(string)

# convert json to dataframe
Extension = pd.DataFrame.from_dict(d, orient='index', columns=['count']).reset_index() # pd.read_json(json_obj, orient ='index')
Extension.rename(columns={"index": "file_extension"},inplace=True)
Extension['reduce'] = round(Extension['count']/100) # scale number of files to plot
print(Extension)


# Plot the waffle Chart
fig = plt.figure(
    figsize = (20, 20),
    FigureClass = Waffle,
    rows = 200,
    values = list(Extension['file_extension']),
    labels = list(Extension['index']),
    title = {
        'label': df3['timestamp'][0],
        'loc': 'center',
        'fontdict': {
            'fontsize': 20
        }
    },
    cmap_name='tab20b',
    legend={
        'loc': 'upper left',
        'bbox_to_anchor': (1, 1),
        'ncol': 3,
        'framealpha': 0,
        'fontsize': 10
    }
)

fig.show()

b. Histogram. Make a histogram using the column `download_ready` from the dataframe `manifest`. 

* Set title to `Broken links`
* Values set to `False` should be labeled as `Broken`.
* Values set to `True` should be labeled as `Not Broken`.
* No legend.
* Set y-label to `Number of links`

In [ ]:
#INSERT CODE HERE
import seaborn as sns
import matplotlib.pyplot as plt

temp_file = 'manifest.tsv'

if Path(temp_file).exists():                         
    df = pd.read_csv(temp_file, sep='\t', low_memory=False)

# Initialize figure size
plt.figure(figsize = ( 6 , 8))
  
# Plot histogram
fig = sns.histplot(data=df, stat='count', x='download_ready',bins=2)
fig.set(xlabel=None)
fig.set(xticklabels=[])
fig.tick_params(bottom=False) 
fig.set(ylabel='Number of links')
fig.set(title='Broken links')

# Display figure
plt.show()

c. Pie chart. Create a pie chart using the values in `status_code` in the dataframe `manifest`. 

* Title should be `Status codes`.
* Add legends.

In [ ]:
#INSERT CODE HERE
# get dataset
file = 'manifest.tsv'
if Path(file).exists():
    df2 = pd.read_csv(file, sep='\t', low_memory=False )

    # Set colors
    cmako = sns.color_palette('bright')

    # Set up data
    status = pd.DataFrame(df['status_code'].value_counts().reset_index())
    # status['isBroken'] = ['Unreachable', 'Reachable']
    print(status)

    # Create pie chart
    plt.pie(data=status,x='status_code',labels='index', autopct='%.1f%%',colors = cmako)
    plt.title('Status code')
    plt.show()
else:
    print(file, ' does not exist!!!')

d. Create a plot using the values in `file_creation_date` in dataframe manifest. However, this exercise is open-ended. 

This is my user-story and your job is to create the best plot you think will show what I want.

* The only part I care about the `file_create_date` is the date. Or months, or years, not sure.
* What I want to do is to create a plot that I can show the increment of data through the years.
* I am still debating whether I should just show how many files are available per year, or do a cumulative plot since it should be, in theory, monotonically increasing.

What do you think? Make a plot, convince me your plot is the best.

In [ ]:
#INSERT CODE HERE
# histogram plot for count of unique 'file_creation_date'
# y_axis 

df['file_creation_date_only'] = df['file_creation_date'][0:10]
# df['file_creation_date_only'] = pd.to_datetime(df3['file_creation_date_only'],errors='coerce')

# Initialize figure size
plt.figure(figsize = ( 6 , 8))
  
# Plot histogram
fig = sns.histplot(data=df, stat='count', x='file_creation_date_only')
fig.set(xlabel='Time')
# fig.set(xticklabels=[])
fig.tick_params(bottom=False) 
fig.set(ylabel='Number of files')
fig.set(title='Number of files across time')

# Display figure
plt.show()